In [1]:
import os, sys
sys.path.append(os.path.abspath("../"))

import pandas as pd
import numpy as np

%load_ext autoreload
%autoreload 2
from config import defaults
from workplaces.workplace import WorkPlace

from calculators.reg_tr_calculator import TR_calculator as RegTrCalc
from calculators.reg_base_flow_rail_matrix_calculator import RegBaseRailFlowCalc
from calculators.reg_base_flow_avia_matrix_calculator import RegBaseAviaFlowCalc
from calculators.reg_dm_calculator import RegDMCalc
from calculators.reg_forecast_potentials_calculator import RegForecastPotentialsCalc
from calculators.matrix_balance_calculator import Fratar

In [2]:
# Рабочая область
wp = WorkPlace(name='rounet_loko',
               config=defaults,
               db_type='mongo')
# Дата провайдер
dp = wp.data_provider

# wp.db.drop_table('test')

### Генерация и сохранение ТР

In [ ]:
# municipalities_df = dp.get_dataset(dataset_name='mun_soc')

# tr_calc = RegTrCalc(municipalities_df)
# tr_calc.generate_tr()

# dp.save_dataset(dataset=tr_calc.trs,
#                 collection_name='trs',
#                 version_name='base')

# trs = dp.get_dataset(dataset_name='trs',
#                      version_name='base')

### Генерация базовой матрицы потоков

In [ ]:
# # ТР
# trs = dp.get_dataset(dataset_name='trs',
#                      version_name='base')

# # ЖД
# flows_rail = dp.get_dataset(dataset_name='flows_rail')
# plos_express_info = dp.get_dataset(dataset_name='plos_express_info')
# # АВИА
# flows_avia = dp.get_dataset(dataset_name='flows_avia')
# plos_avia_info = dp.get_dataset(dataset_name='plos_avia')

In [ ]:
# rail_regbf_calc = RegBaseRailFlowCalc(trs=trs,
#                              flows_base=flows_rail,
#                              plos_express_info=plos_express_info)
# rail_regbf_calc.generate_base_matrix()

# dp.save_dataset(dataset=rail_regbf_calc.save_dict,
#                 collection_name='flow_base_matrix',
#                 version_name='base',
#                 transport_type='rail')

In [ ]:
# avia_regbf_calc = RegBaseAviaFlowCalc(trs=trs,
#                              flows_base=flows_avia,
#                              plos_info=plos_avia_info)
# avia_regbf_calc.generate_base_matrix()

# dp.save_dataset(dataset=avia_regbf_calc.save_dict,
#                 collection_name='flow_base_matrix',
#                 version_name='base',
#                 transport_type='avia')

In [ ]:
# rail_fbm = dp.get_dataset(dataset_name='flow_base_matrix',
#                           version_name='base',
#                           transport_type='rail')
# avia_fbm = dp.get_dataset(dataset_name='flow_base_matrix',
#                           version_name='base',
#                           transport_type='avia')

# fbm = rail_fbm + avia_fbm
# fbm.sum()

### Генерация матрицы расстояний

In [ ]:
# trs = dp.get_dataset(dataset_name='trs',
#                      version_name='base')

# reg_dm_calc = RegDMCalc(trs=trs,
#                         calc_type='by_ort')
# reg_dm_calc.generate_dm()

# dp.save_dataset(dataset=reg_dm_calc.save_dict,
#                 collection_name='dist_matrix',
#                 version_name='base')

# dm = dp.get_dataset(dataset_name='dist_matrix',
#                 version_name='base')

### Расчет матрицы корреспонденций на прогнозируемый год

In [3]:
trs = dp.get_dataset(dataset_name='trs',
                     version_name='base')


# dm = dp.get_dataset(dataset_name='dist_matrix',
#                 version_name='base')


rail_fbm = dp.get_dataset(dataset_name='flow_base_matrix',
                          version_name='base',
                          transport_type='rail')
avia_fbm = dp.get_dataset(dataset_name='flow_base_matrix',
                          version_name='base',
                          transport_type='avia')

elasticities = dp.get_dataset(dataset_name='elasticities')

base_year = 2019
forecast_year = 2025

#### Прогнозные потенциалы

In [4]:
# rail_forecast_potentails_calc
rfpc = RegForecastPotentialsCalc(trs=trs,
                                  base_matrix=rail_fbm,
                                  base_year=base_year,
                                  forecast_year=forecast_year,
                                  tr_type='rail',
                                  elasticities=elasticities)

rfpc.calc_forecast_potentials()

print(rfpc.forecast_attractions.sum(),
      rfpc.forecast_generations.sum())

132083927.62578937 132083927.62578937


In [5]:
# avia_forecast_potentails_calc
afpc = RegForecastPotentialsCalc(trs=trs,
                                  base_matrix=avia_fbm,
                                  base_year=base_year,
                                  forecast_year=forecast_year,
                                  tr_type='avia',
                                  elasticities=elasticities)

afpc.calc_forecast_potentials()

print(afpc.forecast_attractions.sum(),
      afpc.forecast_generations.sum())

200364302.12238008 200364302.12238008


#### Прогнозные матрицы

In [14]:
# ЖД

rail_forecast_attractions = rfpc.forecast_attractions
rail_forecast_generations = rfpc.forecast_generations

# rail_forecast_matrix_calculator
rfmc = Fratar(target_vector_row=rail_forecast_attractions,
              target_vector_col=rail_forecast_generations,
              matrix=rail_fbm)
rfmc.fit()
rail_forecast_matrix = rfmc.matrix

Стартовое среднее квадратичное отклонение по строкам: 32485676477029.188
Стартовое среднее квадратичное отклонение по столбцам: 32169927044167.266


C:\Users\user\rounet_lokotech\calculators\matrix_balance_calculator.py:84: RuntimeWarning: invalid value encountered in true_divide
  f = np.divide(targets, marginals)


Итоговое среднее квадратичная отклонение по строкам: 0.0004023294754057974
Итоговое среднее квадратичная отклонение по столбцам: 1.2982358198477407e-15


In [15]:
# АВИА

avia_forecast_attractions = afpc.forecast_attractions
avia_forecast_generations = afpc.forecast_generations

# avia_forecast_matrix_calculator
afmc = Fratar(target_vector_row=avia_forecast_attractions,
              target_vector_col=avia_forecast_generations,
              matrix=avia_fbm)
afmc.fit()
avia_forecast_matrix = afmc.matrix

Стартовое среднее квадратичное отклонение по строкам: 453787817401719.6
Стартовое среднее квадратичное отклонение по столбцам: 470260100098326.25


C:\Users\user\rounet_lokotech\calculators\matrix_balance_calculator.py:84: RuntimeWarning: divide by zero encountered in true_divide
  f = np.divide(targets, marginals)


Итоговое среднее квадратичная отклонение по строкам: 1747089.5561713078
Итоговое среднее квадратичная отклонение по столбцам: 41.27483756899861
